In [1]:
import pygame as pg
import random as rd
import numpy as np
import time

pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


# File processing

In [155]:
def add_record(filename, name, score):
    with open(filename, 'r') as f:
        strings = f.read().split('\n')
    
    while '' in strings:
        strings.remove('')
    with open(filename, 'w') as file:
        names = []
        for s in strings:
            if len(s) != 0:
                names.append(s.split()[0])
        if name in names:
            i = names.index(name)
            cur_string = strings[i].split()
            if score > int(cur_string[1]):
                strings[i] = cur_string[0] + ' ' + str(score)
        else:
            strings.append(name + ' ' + str(score))
        for s in strings:
            print(s)
            file.write(s + '\n')

# Stuff

In [134]:
def rotate(vector, angle):
    #takes angle in radians and turns vector at it
    A = np.array([
        [np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)]
    ])
    return A@vector

def new_ball(w, h):
    #рисует новый шарик
    rmin, rmax = 10, 100
    x = rd.randint(rmax, w - rmax)
    y = rd.randint(rmax, h - rmax)
    r = rd.randint(rmin, rmax)
    color = COLORS[rd.randint(0, 5)]
    return Ball([x,y], r, color)

def dist(p1, p2):
    return np.sqrt( (p1[0]-p2[0])**2 + (p1[1] - p2[1])**2 )

def new_david(w,h):
    #рисует новую звезду Давида
    rmin, rmax = 10, 100
    x = rd.randint(rmax, w - rmax)
    y = rd.randint(rmax, h - rmax)
    r = rd.randint(rmin, rmax)
    color = COLORS[rd.randint(0, 5)]
    return David([x,y], r, color)

def new_object_on_screen(targets):
    who = 0#rd.randint(0,1)
    if who:
        targets.append(new_ball(w, h))
    else:
        targets.append(new_david(w,h))
    target = targets[len(targets)-1]
    target.change_velocity([rd.randint(-target.rad,target.rad), rd.randint(-target.rad,target.rad)])
    
def finish(s):
    w,h = s.get_size()
    f1 = pg.font.Font(None, 36)
    text1 = f1.render('What\'s your name?', True,
                  (180, 0, 0))
    s.fill((255,255,255))
    s.blit(text1, (w/2, h/2))
    pg.display.update()
    
    player = input("What's your name? ")
    return player

# Class_Target

In [135]:
class Target():
    def __init__(self, kind):
        self.kind = kind
    def change_velocity(self, vel):
        self.velocity = vel

# Class_Ball

In [136]:
class Ball(Target):
    def __init__(self, center, rad, color):
        super().__init__('ball')
        self.center = center #[int, int]
        self.rad = rad #int
        self.velocity = [0,0]
        self.color = color
    def move(self, ampl = 1):
        #ampl < 1 if ball can't make full step (he will collide with wall)
        self.center[0] += ampl*self.velocity[0]
        self.center[1] += ampl*self.velocity[1]
    def draw(self, screen):
        pg.draw.circle(screen, self.color, self.center, self.rad)

# Class_David

In [137]:
class David(Target):
    #act like a Ball, but looks different. In plans - change criteria of pointing on it
    def __init__(self, center, rad, color):
        super().__init__('David')
        self.center = np.array(center) #[int, int]
        self.rad = rad #int
        self.velocity = np.array([0,0])
        self.color = color
        self.points = []
        for i in range(6):
            self.points.append(rotate(np.array([0,self.rad]), i*np.pi/3))
            self.points.append(rotate(np.array([0,self.rad/np.sqrt(3)]), np.pi/6 + i*np.pi/3))
    def move(self, ampl = 1):
        #ampl < 1 if ball can't make full step (he will collide with wall)
        self.center += self.velocity
        for i in range(len(self.points)):
            self.points[i] = rotate(self.points[i], np.pi/8)
    def draw(self, screen):
        draw_points = []
        for p in self.points:
            draw_points.append(p + self.center)
        pg.draw.polygon(screen, self.color, draw_points)

# main

In [14]:
pg.init()

FPS = 2
w, h = 1150, 850
SCORE = 0
screen = pg.display.set_mode((w, h))
clock = pg.time.Clock()

RED = (255, 0, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
GREEN = (0, 255, 0)
MAGENTA = (255, 0, 255)
CYAN = (0, 255, 255)
BLACK = (0, 0, 0)
WHITE = (255,255,255)
COLORS = [RED, BLUE, YELLOW, GREEN, MAGENTA, CYAN]

finished = False
target = new_ball(w,h)
target.change_velocity([rd.randint(-target.rad,target.rad), rd.randint(-target.rad,target.rad)])
target.draw(screen)
pg.display.update()
targets = [target]
count = 0
start_time = time.time()
while not finished:
    cur_time = time.time() - start_time
    clock.tick(FPS)
    count += 1
    for target in targets:
        target.draw(screen)
        #следующие четыре условия реализуют отражение
        if (target.center[0] + target.velocity[0] > w - target.rad):#удар о правую стенку
            target.move((w - target.rad - target.center[0])/abs(target.velocity[0]))
            if target.kind == 'ball':
                target.change_velocity([rd.randint(-target.rad,0), rd.randint(-target.rad,target.rad)])
            elif target.kind == 'David':
                target.change_velocity([-target.velocity[0], target.velocity[1]])
            
        if (target.center[0] + target.velocity[0] < target.rad):#удар о левую стенку
            target.move((- target.rad + target.center[0])/abs(target.velocity[0]))
            if target.kind == 'ball':
                target.change_velocity([rd.randint(0, target.rad), rd.randint(-target.rad,target.rad)])
            elif target.kind == 'David':
                target.change_velocity([-target.velocity[0], target.velocity[1]])
            
        if (target.center[1] + target.velocity[1] > h - target.rad):#удар о нижнюю стенку
            target.move((h - target.rad - target.center[1])/abs(target.velocity[1]))
            if target.kind == 'ball':
                target.change_velocity([rd.randint(-target.rad,target.rad), rd.randint(-target.rad, 0)])
            elif target.kind == 'David':
                target.change_velocity([target.velocity[0], -target.velocity[1]])
            
        if (target.center[1] + target.velocity[1] < target.rad):#удар о верхнюю стенку
            target.move(( - target.rad + target.center[1])/abs(target.velocity[1]))
            if target.kind == 'ball':
                target.change_velocity([rd.randint(-target.rad,target.rad), rd.randint(0, target.rad)])
            elif target.kind == 'David':
                target.change_velocity([target.velocity[0], -target.velocity[1]])
        
        target.move()
    if count == 5: #каждые 5 циклов добавляем в игру мишень
        count = 0
        new_object_on_screen(targets)    
    screen.fill(WHITE)
    for target in targets:
        target.draw(screen)    
    pg.display.update()
    for e in pg.event.get():
        if e.type == pg.MOUSEBUTTONDOWN:
            beaten_number = 0
            add_score = 0
            for target in targets: #проверим не попали ли мы в шарик
                if dist(e.pos, target.center) < target.rad:
                    beaten_number += 1
                    targets.remove(target)
                    if target.kind == 'target':
                        add_score += 1 #за кружок даём очко
                    if target.kind == 'David':
                        add_score += 2 #за звезду Давида - 2 очка
                    #если расскомментировать этот блок, то новые объекты будут добавляться только, если поле пусто
                    #это логично, но куда менее весело
                    '''if len(targets) == 0:
                        new_object_on_screen(targets)'''
                    new_object_on_screen(targets)
            SCORE += add_score*beaten_number #даём больше очков за одновременное попадание по нескольким объектам    
            screen.fill(WHITE) # подготовимся к перерисовыванию шаров
            for target in targets:
                target.draw(screen)
            pg.display.update()
        elif e.type == pg.QUIT:
            pg.quit()
            finished = True
    if cur_time > 30:
        player_name = finish(screen)
        score_table_file_name = 'Records.txt'
        add_record(score_table_file_name, player_name, SCORE)
        pg.quit()
        finished = True


What's your name? Peter


# Debug

In [151]:
f = open('Records.txt', 'w+')
f.write('oleg 13\n')
f.write('anna 14\n')
f.close()

In [152]:
add_record('Records.txt', 'peter', 10)

oleg 13
anna 14
peter 10


In [153]:
add_record('Records.txt', 'anna', 9)

oleg 13
anna 14
peter 10


In [154]:
add_record('Records.txt', 'peter', 20)

oleg 13
anna 14
peter20
